In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.layers import Dense, Dropout, Flatten

import warnings
warnings.filterwarnings("ignore")
import shelve

Using TensorFlow backend.


# Read data

In [2]:
df = pd.read_csv("my_train_thin_thin.csv")
y = df["is_churn"]
df.drop(["is_churn","msno"],axis=1,inplace=True)

In [3]:
df["col_diff_last"] = df["date_count_201702"]-df["date_count_201701"]
df["col_diff_second_last"] = df["date_count_201701"]-df["date_count_201612"]
df["date_diff"] = 20170301-df["max_membership_expire_date"]

In [4]:
df = df[['max_membership_expire_date',
 'date_diff',
 'is_auto_renew',
 'last_is_cancel',
 'last_is_auto_renew',
 'payment_plan_days',
 'trans_count',
 'is_cancel',
 'col_diff1',
 'date_count_201703',
 'total_secs_mean_201703',
 'num_unq_mean_201703',
 'slope',
 'registered_via_7',
 'num_100_pct_201703',
 'total_secs_mean_201702',
 'num_25_pct_201703',
 'num_50_pct_201703',
 'num_985_pct_201703',
 'num_75_pct_201703',
 'num_unq_mean_201609',
 'num_unq_mean_201702',
 'num_985_pct_201702',
 'num_unq_mean_201701',
 'num_25_pct_201702',
 'total_secs_mean_201701',
 'col_diff_last',
 'num_50_pct_201702',
 'num_100_pct_201702',
 'num_75_pct_201702',
 'num_25_pct_201701',
 'date_count_201702',
 'col_diff２',
 'total_secs_mean_201612',
 'num_50_pct_201701',
 'num_75_pct_201701',
 'num_985_pct_201701',
 'date_count_201701',
 'num_100_pct_201701',
 'num_unq_mean_201612',
 'num_25_pct_201612',
 'num_50_pct_201612',
 'num_unq_mean_201611',
 'num_25_pct_201611',
 'col_diff_second_last',
 'total_secs_mean_201610',
 'num_100_pct_201609',
 'num_985_pct_201612',
 'num_unq_mean_201610',
 'total_secs_mean_201611']]

# train test split

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, random_state=1)

In [ ]:
y_train = np_utils.to_categorical(y_train, 2)
y_test = np_utils.to_categorical(y_test, 2)

# preprocess

In [6]:
def cap_it(d_cap,X):
    for k in d_cap.keys():
        cap = d_cap[k]
        X[k][X[k]>cap] = cap
    return X

In [6]:
# cap
d_cap ={}
for col in X_train.columns.tolist():
    if col[:5] == "total":
        cap = np.percentile(X_train[col], 70)
        X_train[col][X_train[col]>cap] = cap
        d_cap[col] = cap
        
def cap_it(d_cap,X):
    for k in d_cap.keys():
        cap = d_cap[k]
        X[k][X[k]>cap] = cap
    return X

X_train = X_train.values
X_test = cap_it(d_cap,X_test)
X_test = X_test.values

In [6]:
# min-max-scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# XGB Model

In [ ]:
from xgboost import XGBClassifier

In [ ]:
help(XGBClassifier)

In [ ]:
# param_dist = {'objective':'binary:logistic'} 0.12251
# param_dist = {'objective':'binary:logistic',"max_depth":4,"n_estimators":500} 0.12957
# param_dist = {'objective':'binary:logistic',"max_depth":5,"n_estimators":100,"min_child_weight":1.5} 0.12426
# param_dist = {'objective':'binary:logistic',"max_depth":5,"n_estimators":100,"min_child_weight":2} 0.12385

In [7]:
from xgboost import XGBClassifier
param_dist = {'objective':'binary:logistic'
             }
clf = XGBClassifier(**param_dist)
clf.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric='logloss',
        verbose=True)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	validation_0-logloss:0.613504	validation_1-logloss:0.613437
[1]	validation_0-logloss:0.548368	validation_1-logloss:0.548236
[2]	validation_0-logloss:0.49424	validation_1-logloss:0.494069
[3]	validation_0-logloss:0.447993	validation_1-logloss:0.447813
[4]	validation_0-logloss:0.408578	validation_1-logloss:0.408397
[5]	validation_0-logloss:0.374968	validation_1-logloss:0.374809
[6]	validation_0-logloss:0.346033	validation_1-logloss:0.345885
[7]	validation_0-logloss:0.320343	validation_1-logloss:0.32023
[8]	validation_0-logloss:0.298489	validation_1-logloss:0.298389
[9]	validation_0-logloss:0.278992	validation_1-logloss:0.278926
[10]	validation_0-logloss:0.261961	validation_1-logloss:0.261923
[11]	validation_0-logloss:0.247227	validation_1-logloss:0.247198
[12]	validation_0-logloss:0.234016	validation_1-logloss:0.23403
[13]	validation_0-logloss:0.222518	validation_1-logloss:0.222522
[14]	validation_0-logloss:0.212025	validation_1-logloss:0.212085
[15]	validation_0-logloss:0.2027	valid

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [9]:
import shelve
filename="shelve.out"
my_shelf = shelve.open(filename,"n") # "n" for new

my_shelf["scaler"] = globals()["scaler"]
my_shelf["clf"] = globals()["clf"]
 
my_shelf.close()

# Test data

In [ ]:
# load model

In [ ]:
import shelve
filename="shelve.out"
my_shelf = shelve.open(filename)
 
keys = my_shelf.keys()
 
# import all the variables
for key in keys:
    try:
        globals()[key]=my_shelf[key]
        print("loading:    " + str(type(my_shelf[key])) + "\t"+ key)
    except:
        print("")
          
my_shelf.close()

In [9]:
df = pd.read_csv("my_test_thin_thin.csv")
y = df["msno"]
df.drop("msno",axis=1,inplace=True)
df["col_diff_last"] = df["date_count_201703"]-df["date_count_201702"]
df["col_diff_second_last"] = df["date_count_201702"]-df["date_count_201701"]
df["date_diff"] = 20170401-df["max_membership_expire_date"]
df = df[['max_membership_expire_date',
 'date_diff',
 'is_auto_renew',
 'last_is_cancel',
 'last_is_auto_renew',
 'payment_plan_days',
 'trans_count',
 'is_cancel',
 'col_diff1',
 'date_count_201703',
 'total_secs_mean_201703',
 'num_unq_mean_201703',
 'slope',
 'registered_via_7',
 'num_100_pct_201703',
 'total_secs_mean_201702',
 'num_25_pct_201703',
 'num_50_pct_201703',
 'num_985_pct_201703',
 'num_75_pct_201703',
 'num_unq_mean_201609',
 'num_unq_mean_201702',
 'num_985_pct_201702',
 'num_unq_mean_201701',
 'num_25_pct_201702',
 'total_secs_mean_201701',
 'col_diff_last',
 'num_50_pct_201702',
 'num_100_pct_201702',
 'num_75_pct_201702',
 'num_25_pct_201701',
 'date_count_201702',
 'col_diff２',
 'total_secs_mean_201612',
 'num_50_pct_201701',
 'num_75_pct_201701',
 'num_985_pct_201701',
 'date_count_201701',
 'num_100_pct_201701',
 'num_unq_mean_201612',
 'num_25_pct_201612',
 'num_50_pct_201612',
 'num_unq_mean_201611',
 'num_25_pct_201611',
 'col_diff_second_last',
 'total_secs_mean_201610',
 'num_100_pct_201609',
 'num_985_pct_201612',
 'num_unq_mean_201610',
 'total_secs_mean_201611']]


In [10]:
X_test_for_real = df.values
X_test_for_real = scaler.transform(X_test_for_real)

In [11]:
ret = clf.predict_proba(X_test_for_real)

In [22]:
df_ret = pd.DataFrame({"msno":y, "is_churn":ret[:,1]})
df_ret.to_csv("ret.csv",index=False)

In [23]:
df_ret[df_ret.is_churn<0.5].shape[0]/df_ret.shape[0]

0.9934653559177098

In [25]:
df_ret["is_churn"][df_ret.is_churn < 0.3] = df_ret.is_churn - 0.05

In [26]:
df_ret["is_churn"][df_ret.is_churn > 0.9] = df_ret.is_churn + 0.02

In [27]:
df_ret["is_churn"][df_ret.is_churn > 1] = 0.99

In [28]:
df_ret["is_churn"][df_ret.is_churn < 0 ] = 0.01

In [29]:
df_ret.to_csv("ret1.csv",index=False)

In [ ]:
from keras.models import load_model
import pandas as pd
model = load_model("model.h5")

In [ ]:
# preprocess test set

In [ ]:
b

In [ ]:
# predict and  save

In [ ]:
ret = model.predict(X_test_for_real)
df_ret = pd.DataFrame({"msno":y, "is_churn":ret[:,1].tolist()})
df_ret.to_csv("ret.csv",index=False)

In [ ]:
df_ret

In [ ]:
#96% 0 in the data 